In [1]:
import pandas as pd
import numpy as np

In [52]:
df = pd.read_csv(r'C:\Users\jakep\OneDrive\Documents\CS639\citibike-network\Data\citibike.csv',
                parse_dates = ['starttime'],
                usecols = ['starttime', 'start station id', 'start station name', 'end station id','start station latitude', 'start station longitude', 'end station latitude', 'end station longitude', 'end station name'])
print(df)

                      starttime  start station id  \
0       2019-06-01 00:00:01.500              3602   
1       2019-06-01 00:00:04.240              3054   
2       2019-06-01 00:00:06.019               229   
3       2019-06-01 00:00:06.776              3771   
4       2019-06-01 00:00:07.520               441   
...                         ...               ...   
2125365 2019-06-30 23:59:32.892              3113   
2125366 2019-06-30 23:59:33.699               445   
2125367 2019-06-30 23:59:47.405              3307   
2125368 2019-06-30 23:59:51.598               402   
2125369 2019-06-30 23:59:54.367              3788   

                     start station name  start station latitude  \
0                        31 Ave & 34 St               40.763154   
1               Greene Ave & Throop Ave               40.689493   
2                        Great Jones St               40.727434   
3               McKibbin St & Bogart St               40.706237   
4                       E 52

In [53]:
stations = df.groupby(['start station id', 'start station name', 'start station latitude', 'start station longitude']).size()
stations = stations.reset_index()
stations

,start station id,start station name,start station latitude,start station longitude,0
0,72,W 52 St & 11 Ave,40.767272,-73.993929,4849
1,79,Franklin St & W Broadway,40.719116,-74.006667,2980
2,82,St James Pl & Pearl St,40.711174,-74.000165,1403
3,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,1962
4,116,W 17 St & 8 Ave,40.741776,-74.001497,6839
...,...,...,...,...,...
788,3787,Dock 72 Way & Market St,40.699850,-73.971410,516
789,3788,E 12 St & 4 Av,40.732690,-73.989740,3538
790,3790,Henry St & Middagh St,40.700300,-73.991581,1644
791,3793,Stuyvesant Walk & 1 Av Loop,40.732340,-73.978984,111


In [54]:
import shapefile
from shapely.geometry import shape, Point

r = shapefile.Reader(r"C:\Users\jakep\Downloads\Borough Boundaries\geo_export_43b35747-351a-4fbf-944b-eaee1f68385e.shp")
shapes = r.shapes()

polygon = shape(shapes[1])    

def check(lon, lat):
    point = Point(lon, lat)
    return polygon.contains(point)

stations['Manhattan'] = stations.apply(lambda x: check(x['start station longitude'], x['start station latitude']), axis=1)
stations

,start station id,start station name,start station latitude,start station longitude,0,Manhattan
0,72,W 52 St & 11 Ave,40.767272,-73.993929,4849,True
1,79,Franklin St & W Broadway,40.719116,-74.006667,2980,True
2,82,St James Pl & Pearl St,40.711174,-74.000165,1403,True
3,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,1962,False
4,116,W 17 St & 8 Ave,40.741776,-74.001497,6839,True
...,...,...,...,...,...,...
788,3787,Dock 72 Way & Market St,40.699850,-73.971410,516,False
789,3788,E 12 St & 4 Av,40.732690,-73.989740,3538,True
790,3790,Henry St & Middagh St,40.700300,-73.991581,1644,False
791,3793,Stuyvesant Walk & 1 Av Loop,40.732340,-73.978984,111,True


In [55]:
manhattan_stations = stations[stations['Manhattan']].copy()
manhattan_stations_ids = list(manhattan_stations['start station id'])
print(len(manhattan_stations_ids))
manhattan_stations

419


,start station id,start station name,start station latitude,start station longitude,0,Manhattan
0,72,W 52 St & 11 Ave,40.767272,-73.993929,4849,True
1,79,Franklin St & W Broadway,40.719116,-74.006667,2980,True
2,82,St James Pl & Pearl St,40.711174,-74.000165,1403,True
4,116,W 17 St & 8 Ave,40.741776,-74.001497,6839,True
7,127,Barrow St & Hudson St,40.731724,-74.006744,6544,True
...,...,...,...,...,...,...
758,3747,E 84 St & 3 Ave,40.777554,-73.955128,3242,True
759,3749,Lexington Ave & E 36 St,40.747574,-73.978801,2792,True
787,3783,Cliff St & Fulton St,40.708380,-74.004950,1908,True
789,3788,E 12 St & 4 Av,40.732690,-73.989740,3538,True


### Remove loops
We are not interested in rides that start and end at the same station.

In [56]:
same_station = df['start station id'] == df['end station id']
print(f'{sum(same_station)}')
df = df[~same_station]

53758


In [57]:
df.dtypes
df.loc[df['starttime'].dt.dayofweek < 5, 'Weekday'] = 'Weekday'
df.loc[df['starttime'].dt.dayofweek >= 5, 'Weekday'] = 'Weekend'
#df['Weekday'] = df['starttime'].dt.dayofweek < 5 else 'Weekend'
df.loc[(12 > df['starttime'].dt.hour) & (df['starttime'].dt.hour >= 7), 'Daytime'] = 'Morning'
df.loc[(21 > df['starttime'].dt.hour) & (df['starttime'].dt.hour >= 12), 'Daytime'] = 'Afternoon'
df.loc[(21 <= df['starttime'].dt.hour) | (df['starttime'].dt.hour < 7), 'Daytime'] = 'Night'
#df['Daytime'] = (9 > df['starttime'].dt.hour) & (df['starttime'].dt.hour >= 7)
#df['Daytime'] = (9 > df['starttime'].dt.hour) & (df['starttime'].dt.hour >= 7)
df

,starttime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,Weekday,Daytime
0,2019-06-01 00:00:01.500,3602,31 Ave & 34 St,40.763154,-73.920827,3570,35 Ave & 37 St,40.755733,-73.923661,Weekend,Night
1,2019-06-01 00:00:04.240,3054,Greene Ave & Throop Ave,40.689493,-73.942061,3781,Greene Av & Myrtle Av,40.698568,-73.918877,Weekend,Night
2,2019-06-01 00:00:06.019,229,Great Jones St,40.727434,-73.993790,326,E 11 St & 1 Ave,40.729538,-73.984267,Weekend,Night
3,2019-06-01 00:00:06.776,3771,McKibbin St & Bogart St,40.706237,-73.933871,3016,Kent Ave & N 7 St,40.720368,-73.961651,Weekend,Night
4,2019-06-01 00:00:07.520,441,E 52 St & 2 Ave,40.756014,-73.967416,3159,W 67 St & Broadway,40.774925,-73.982666,Weekend,Night
...,...,...,...,...,...,...,...,...,...,...,...
2125365,2019-06-30 23:59:32.892,3113,Greenpoint Ave & Manhattan Ave,40.730260,-73.953940,3107,Bedford Ave & Nassau Ave,40.723117,-73.952123,Weekend,Night
2125366,2019-06-30 23:59:33.699,445,E 10 St & Avenue A,40.727408,-73.981420,3737,Stanton St & Norfolk St,40.720747,-73.986274,Weekend,Night
2125367,2019-06-30 23:59:47.405,3307,West End Ave & W 94 St,40.794165,-73.974124,3320,Central Park West & W 100 St,40.794067,-73.962868,Weekend,Night
2125368,2019-06-30 23:59:51.598,402,Broadway & E 22 St,40.740343,-73.989551,400,Pitt St & Stanton St,40.719261,-73.981780,Weekend,Night


In [58]:
# Given a point(lon, lat) determine whether is it located in Manhattan
def station_in_city(start, stop):
    s = start in manhattan_stations_ids
    e = stop in manhattan_stations_ids
    return s and e
    
df['Manhattan'] = df.apply(lambda x: station_in_city(x['start station id'], x['end station id']), axis = 1)

In [59]:
# Percentage of rides starting and ending in Manhattan
np.mean(df['Manhattan'])

0.744229131709992

In [60]:
# Only keep stations in Manhattan
df = df[df['Manhattan']]
df

,starttime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,Weekday,Daytime,Manhattan
2,2019-06-01 00:00:06.019,229,Great Jones St,40.727434,-73.993790,326,E 11 St & 1 Ave,40.729538,-73.984267,Weekend,Night,True
4,2019-06-01 00:00:07.520,441,E 52 St & 2 Ave,40.756014,-73.967416,3159,W 67 St & Broadway,40.774925,-73.982666,Weekend,Night,True
5,2019-06-01 00:00:07.986,3236,W 42 St & Dyer Ave,40.758985,-73.993800,495,W 47 St & 10 Ave,40.762699,-73.993012,Weekend,Night,True
7,2019-06-01 00:00:14.644,3467,W Broadway & Spring Street,40.724947,-74.001659,401,Allen St & Rivington St,40.720196,-73.989978,Weekend,Night,True
8,2019-06-01 00:00:18.197,379,W 31 St & 7 Ave,40.749156,-73.991600,2006,Central Park S & 6 Ave,40.765909,-73.976342,Weekend,Night,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2125364,2019-06-30 23:59:32.080,514,12 Ave & W 40 St,40.760875,-74.002777,458,11 Ave & W 27 St,40.751396,-74.005226,Weekend,Night,True
2125366,2019-06-30 23:59:33.699,445,E 10 St & Avenue A,40.727408,-73.981420,3737,Stanton St & Norfolk St,40.720747,-73.986274,Weekend,Night,True
2125367,2019-06-30 23:59:47.405,3307,West End Ave & W 94 St,40.794165,-73.974124,3320,Central Park West & W 100 St,40.794067,-73.962868,Weekend,Night,True
2125368,2019-06-30 23:59:51.598,402,Broadway & E 22 St,40.740343,-73.989551,400,Pitt St & Stanton St,40.719261,-73.981780,Weekend,Night,True


In [61]:
summary = df.groupby(['start station id', 'start station name', 'end station name', 'end station id', 'start station latitude', 'start station longitude', 'end station latitude', 'end station longitude', 'Weekday', 'Daytime']).size()
summary = summary.unstack(fill_value=0).unstack(fill_value=0)
summary.columns = [' '.join(col).strip() for col in summary.columns.values]
summary

Afternoon Weekday  \
start station id start station name          end station name              end station id start station latitude start station longitude end station latitude end station longitude                      
72               W 52 St & 11 Ave            1 Ave & E 16 St               504            40.767272              -73.993929              40.732219            -73.981656                             1   
                                             1 Ave & E 18 St               2003           40.767272              -73.993929              40.733812            -73.980544                             1   
                                             1 Ave & E 30 St               536            40.767272              -73.993929              40.741444            -73.975361                             1   
                                             1 Ave & E 62 St               3142           40.767272              -73.993929              40.761227            -73.960940                             4   
                                             1 Ave & E 68 St               3141           40.767272              -73.993929              40.765005            -73.958185                             0   
...                                                                                                                                                                                                ...   
3793             Stuyvesant Walk & 1 Av Loop W 34 St & 11 Ave              525            40.732340              -73.978984              40.755942            -74.002116                             0   
                                             W 37 St & 5 Ave               485            40.732340              -73.978984              40.750380            -73.983390                             0   
                                             W 38 St & 8 Ave               523            40.732340              -73.978984              40.754666            -73.991382                             0   
                                             Washington Pl & 6 Ave         369            40.732340              -73.978984              40.732241            -74.000264                             0   
                                             Washington St & Gansevoort St 405            40.732340              -73.978984              40.739323            -74.008119                             0   

                                                                                                                                                                                     Afternoon Weekend  \
start station id start station name          end station name              end station id start station latitude start station longitude end station latitude end station longitude                      
72               W 52 St & 11 Ave            1 Ave & E 16 St               504            40.767272              -73.993929              40.732219            -73.981656                             0   
                                             1 Ave & E 18 St               2003           40.767272              -73.993929              40.733812            -73.980544                             1   
                                             1 Ave & E 30 St               536            40.767272              -73.993929              40.741444            -73.975361                             0   
                                             1 Ave & E 62 St               3142           40.767272              -73.993929              40.761227            -73.960940                             0   
                                             1 Ave & E 68 St               3141           40.767272              -73.993929              40.765005            -73.958185                             0   
...                                                                                                                                         

In [62]:
summary['Morning'] = summary['Morning Weekday'] + summary['Morning Weekend']
summary['Afternoon'] = summary['Afternoon Weekday'] + summary['Afternoon Weekend']
summary['Night'] = summary['Night Weekday'] + summary['Night Weekend']
summary['Weekday'] = summary['Night Weekday'] + summary['Morning Weekday'] + summary['Afternoon Weekday']
summary['Weekend'] = summary['Night Weekend'] + summary['Morning Weekend'] + summary['Afternoon Weekend']
summary['Total'] = summary['Morning'] + summary['Afternoon'] +summary['Night']
summary

Afternoon Weekday  \
start station id start station name          end station name              end station id start station latitude start station longitude end station latitude end station longitude                      
72               W 52 St & 11 Ave            1 Ave & E 16 St               504            40.767272              -73.993929              40.732219            -73.981656                             1   
                                             1 Ave & E 18 St               2003           40.767272              -73.993929              40.733812            -73.980544                             1   
                                             1 Ave & E 30 St               536            40.767272              -73.993929              40.741444            -73.975361                             1   
                                             1 Ave & E 62 St               3142           40.767272              -73.993929              40.761227            -73.960940                             4   
                                             1 Ave & E 68 St               3141           40.767272              -73.993929              40.765005            -73.958185                             0   
...                                                                                                                                                                                                ...   
3793             Stuyvesant Walk & 1 Av Loop W 34 St & 11 Ave              525            40.732340              -73.978984              40.755942            -74.002116                             0   
                                             W 37 St & 5 Ave               485            40.732340              -73.978984              40.750380            -73.983390                             0   
                                             W 38 St & 8 Ave               523            40.732340              -73.978984              40.754666            -73.991382                             0   
                                             Washington Pl & 6 Ave         369            40.732340              -73.978984              40.732241            -74.000264                             0   
                                             Washington St & Gansevoort St 405            40.732340              -73.978984              40.739323            -74.008119                             0   

                                                                                                                                                                                     Afternoon Weekend  \
start station id start station name          end station name              end station id start station latitude start station longitude end station latitude end station longitude                      
72               W 52 St & 11 Ave            1 Ave & E 16 St               504            40.767272              -73.993929              40.732219            -73.981656                             0   
                                             1 Ave & E 18 St               2003           40.767272              -73.993929              40.733812            -73.980544                             1   
                                             1 Ave & E 30 St               536            40.767272              -73.993929              40.741444            -73.975361                             0   
                                             1 Ave & E 62 St               3142           40.767272              -73.993929              40.761227            -73.960940                             0   
                                             1 Ave & E 68 St               3141           40.767272              -73.993929              40.765005            -73.958185                             0   
...                                                                                                                                         

In [63]:
cols = list(summary.columns)
summary = summary.reset_index()

In [64]:
for col in cols:
    summary.sort_values(col, inplace=True, ascending=False)
    summary[f'{col}_R'] = range(len(summary))

In [65]:
ranks = [col for col in summary if col.endswith('_R')]
ranks

['Afternoon Weekday_R',
 'Afternoon Weekend_R',
 'Morning Weekday_R',
 'Morning Weekend_R',
 'Night Weekday_R',
 'Night Weekend_R',
 'Morning_R',
 'Afternoon_R',
 'Night_R',
 'Weekday_R',
 'Weekend_R',
 'Total_R']

In [66]:
# Only keep rides where one of the combinations ranks in 1000 most frequent
summary = summary[(summary[ranks] < 1000).any(axis=1)]

In [67]:
stations = stations[stations.Manhattan]
stations

,start station id,start station name,start station latitude,start station longitude,0,Manhattan
0,72,W 52 St & 11 Ave,40.767272,-73.993929,4849,True
1,79,Franklin St & W Broadway,40.719116,-74.006667,2980,True
2,82,St James Pl & Pearl St,40.711174,-74.000165,1403,True
4,116,W 17 St & 8 Ave,40.741776,-74.001497,6839,True
7,127,Barrow St & Hudson St,40.731724,-74.006744,6544,True
...,...,...,...,...,...,...
758,3747,E 84 St & 3 Ave,40.777554,-73.955128,3242,True
759,3749,Lexington Ave & E 36 St,40.747574,-73.978801,2792,True
787,3783,Cliff St & Fulton St,40.708380,-74.004950,1908,True
789,3788,E 12 St & 4 Av,40.732690,-73.989740,3538,True


###  Export files to JSON

In [47]:
#summary.to_json('paths.json', orient='index', indent=4)

In [19]:
#stations.drop([0, 'Manhattan'], axis=1).set_index('start station id').to_json('stations.json', orient='records', indent=4)

In [72]:
import json
export = {'nodes': stations.drop([0, 'Manhattan'], axis=1).to_dict(orient='records'),
         'links': summary.to_dict(orient='records')}
json.dumps(export, indent=4)

with open('combined.json', 'w') as json_file:
  json.dump(export, json_file, indent=4)

In [71]:
summary.to_dict(orient='records')

[{'start station id': 3182,
  'start station name': 'Yankee Ferry Terminal',
  'end station name': 'Soissons Landing',
  'end station id': 3254,
  'start station latitude': 40.686931,
  'start station longitude': -74.016966,
  'end station latitude': 40.69231660719192,
  'end station longitude': -74.01486575603485,
  'Afternoon Weekday': 169,
  'Afternoon Weekend': 581,
  'Morning Weekday': 10,
  'Morning Weekend': 94,
  'Night Weekday': 2,
  'Night Weekend': 0,
  'Morning': 104,
  'Afternoon': 750,
  'Night': 2,
  'Weekday': 181,
  'Weekend': 675,
  'Total': 856,
  'Afternoon Weekday_R': 42,
  'Afternoon Weekend_R': 0,
  'Morning Weekday_R': 7506,
  'Morning Weekend_R': 3,
  'Night Weekday_R': 18470,
  'Night Weekend_R': 87670,
  'Morning_R': 78,
  'Afternoon_R': 1,
  'Night_R': 25971,
  'Weekday_R': 143,
  'Weekend_R': 0,
  'Total_R': 0},
 {'start station id': 3254,
  'start station name': 'Soissons Landing',
  'end station name': 'Picnic Point',
  'end station id': 3479,
  'start st